In [4]:
from random import Random
from Crypto.Util.number import bytes_to_long, long_to_bytes
import math

r = Random()

len_bts = 10000
bts = r.randbytes(len_bts)

In [7]:
# Frequency test

# returns true for random sequences
def frequency_test(b: bytes) -> bool:
    m = bytes_to_long(b)
    n = len(b) * 8

    converted = [0] * n
    for i in range(n):
        converted[i] = m % 2
        m = m << 1
        if converted[i] == 0:
            converted[i] = -1

    s = sum(converted)

    sobs = abs(s) / (n ** 0.5)

    p = math.erfc(sobs / (2 ** 0.5))

    return p >= 0.01


-9998
0.5270892568875369


In [ ]:
# Block frequency test

def block_test(b: bytes) -> bool:
